In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from PIL import Image
import random
import math

Using TensorFlow backend.


In [2]:
training_data = np.load('./output/training_data.npy')
training_labels = np.load('./output/training_dists.npy')

#pad the training data matrix so it is 100x1024

#(500, 100, 1000, 1)

zero_padding_array = np.zeros((500,100,600,1))

In [3]:
training_data.shape

(500, 100, 1000, 1)

In [4]:
training_labels.shape

(500, 100, 100, 1)

In [5]:
zero_padding_array.shape

(500, 100, 600, 1)

In [6]:
print(training_data.shape)
training_data = np.concatenate([training_data, zero_padding_array], axis=2)
print(training_data.shape)

print(training_labels.shape)

samples_count = training_data.shape[0]

train_size = math.floor(0.85*training_data.shape[0])

shuffle_indices = random.sample(range(0, samples_count), samples_count)

print('train_size: ', train_size)
print('samples_count: ', samples_count)
indices_train = shuffle_indices[0:train_size]
indices_test = shuffle_indices[train_size:samples_count]

X_train = training_data[indices_train,:]
Y_train = training_labels[indices_train]

X_test = training_data[indices_test,:]
Y_test = training_labels[indices_test]

print('X_train.shape : ', X_train.shape)
print('X_test.shape : ', X_test.shape)

print('Y_train.shape : ', Y_train.shape)
print('Y_test.shape : ', Y_test.shape)

np.save('X_test.npy', X_test)
np.save('Y_test.npy', Y_test)

model = Sequential()

# Layer 1
model.add(Conv2D(64, input_shape=(100,1600,1), kernel_size=(2,1), strides=(1,1), padding='same'))
#model.add(Conv2D(48, input_shape=(400,11,1), kernel_size=(2,2), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))

# Layer 2
model.add(Conv2D(32, (2, 1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))

# Layer 3
model.add(Conv2D(16, (2, 1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))

# Layer 4
model.add(Conv2D(1, (2, 1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))

model.summary()

(500, 100, 1000, 1)
(500, 100, 1600, 1)
(500, 100, 100, 1)
train_size:  425
samples_count:  500
X_train.shape :  (425, 100, 1600, 1)
X_test.shape :  (75, 100, 1600, 1)
Y_train.shape :  (425, 100, 100, 1)
Y_test.shape :  (75, 100, 100, 1)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 1600, 64)     192       
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 1600, 64)     256       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 1600, 64)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 800, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 800, 32)      4128      
______________

In [7]:
# (4) Compile 
#model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])

In [9]:
# (5) Train
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=16, epochs=4, verbose=1)

model.save('./output/CNN_model_03.h5')

Train on 425 samples, validate on 75 samples
Epoch 1/4
425/425 [==============================] - 150s 352ms/step - loss: 0.2653 - accuracy: 0.6451 - val_loss: 0.2098 - val_accuracy: 0.6557
Epoch 2/4
425/425 [==============================] - 153s 360ms/step - loss: 0.2313 - accuracy: 0.6450 - val_loss: 0.2215 - val_accuracy: 0.6557
Epoch 3/4
425/425 [==============================] - 157s 369ms/step - loss: 0.2171 - accuracy: 0.6452 - val_loss: 0.2435 - val_accuracy: 0.6557
Epoch 4/4
425/425 [==============================] - 163s 385ms/step - loss: 0.2143 - accuracy: 0.6453 - val_loss: 0.2524 - val_accuracy: 0.6557


In [11]:
training_result = model.evaluate(X_train, Y_train)
testing_result = model.evaluate(X_test, Y_test)

75/75 [==============================] - 7s 99ms/step
Training Acc:  [0.2648667133205077, 0.6456983685493469]
Testing Acc:  [0.2524347738424937, 0.6556559801101685]


In [12]:
print('Training Acc: ', training_result[1])
print('Testing Acc: ', testing_result[1])

Training Acc:  0.6456983685493469
Testing Acc:  0.6556559801101685
